
# ⚙️ SciPy — Calcul scientifique appliqué
_Master IA-GI — Notebook 5_

Ce notebook suit la structure de votre support **SciPy** et met l'accent sur les sous-modules clés :  
`scipy.linalg`, `scipy.optimize`, `scipy.integrate`, `scipy.interpolate`, `scipy.stats`, `scipy.signal`, `scipy.fft`, `scipy.spatial`, `scipy.ndimage`.

**Objectifs d’apprentissage**
- Résoudre des problèmes classiques de **calcul scientifique** avec SciPy
- Comprendre quand utiliser **SciPy vs NumPy**
- Appliquer **optimisation**, **intégration**, **interpolation**, **statistiques**, **signal/FFT**, **spatial**, **imagerie**
- Construire un mini‑projet intégrant plusieurs briques SciPy

**Pré-requis** : NumPy (Notebook 3) + Matplotlib (Notebook 4)  
**Durée estimée** : 3–4h



---
## 0) ⚙️ Préparation


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

import scipy
from scipy import linalg, optimize, integrate, interpolate, stats, signal, fft, spatial, ndimage

print("NumPy:", np.__version__)
print("SciPy:", scipy.__version__)



---
## 1) `scipy.linalg` vs `numpy.linalg`
### 1.1 Décompositions & solveurs


In [ ]:

from scipy.linalg import lu, lu_factor, lu_solve, cholesky, qr, svd

A = np.array([[4., 2., 0.],
              [2., 4., 2.],
              [0., 2., 4.]])
b = np.array([2., 4., 6.])

P, L, U = lu(A)
lu_piv = lu_factor(A)
x_lu = lu_solve(lu_piv, b)

Lchol = cholesky(A, lower=True)
Q, R = qr(A)
U_s, S_s, Vt_s = svd(A)

x_lu, np.allclose(A@x_lu, b), np.allclose(A, Q@R)



**Exercice 1.1 — Conditionnement**
<details>
<summary>✅ Solution</summary>

```python
from numpy.linalg import norm
condA = np.linalg.cond(A)
rng = np.random.default_rng(0)
noise = rng.normal(0, 1e-4, size=A.shape)
x0 = np.linalg.solve(A, b)
x1 = np.linalg.solve(A + noise, b)
relative_change = norm(x1 - x0) / (norm(x0) + 1e-12)
condA, relative_change
```
</details>



---
## 2) `scipy.optimize` — Optimisation
### 2.1 Minimisation non contrainte


In [ ]:

from scipy.optimize import minimize

def f(x):
    return (x[0]-1)**2 + 5*(x[1]-2)**2

x0 = np.array([0.0, 0.0])
res = minimize(f, x0, method="BFGS")
res.x, res.fun, res.success



### 2.2 Contraintes & bornes


In [ ]:

from scipy.optimize import Bounds, LinearConstraint

def f2(v):
    x, y = v
    return (x-1)**2 + (y-2)**2

bounds = Bounds([0,0], [np.inf, np.inf])
A = np.array([[1,1]])
lc = LinearConstraint(A, [3], [3])

res2 = minimize(f2, x0=[0.1, 2.9], method="trust-constr", bounds=bounds, constraints=[lc])
res2.x, res2.fun, res2.success



### 2.3 Recherche de racines


In [ ]:

from scipy.optimize import root_scalar

g = lambda x: np.cos(x) - x
sol = root_scalar(g, bracket=[0, 1])
sol.root, sol.converged



---
## 3) `scipy.integrate` — Intégration & EDO
### 3.1 Quadrature simple


In [ ]:

from scipy.integrate import quad
h = lambda x: np.sin(x) / x if x != 0 else 1.0
I, err = quad(h, 0, np.inf, limit=200)
I, err



### 3.2 EDO avec `solve_ivp`


In [ ]:

from scipy.integrate import solve_ivp

def f_ode(t, y):
    return -y + np.sin(t)

sol = solve_ivp(f_ode, t_span=(0, 10), y0=[1.0], t_eval=np.linspace(0,10,200))
plt.figure()
plt.plot(sol.t, sol.y[0])
plt.title("EDO: y'=-y+sin(t)")
plt.xlabel("t"); plt.ylabel("y(t)")
plt.tight_layout()



---
## 4) `scipy.interpolate` — Interpolation
### 4.1 Interp 1D


In [ ]:

from scipy.interpolate import interp1d

x = np.linspace(0, 10, 11)
y = np.sin(x)
f_lin = interp1d(x, y, kind="linear")
xx = np.linspace(0, 10, 200)
plt.figure()
plt.plot(x, y, 'o')
plt.plot(xx, f_lin(xx))
plt.title("Interpolation 1D (linéaire)")
plt.xlabel("x"); plt.ylabel("y")
plt.tight_layout()


In [ ]:

# Cubique (graphe séparé)
f_cub = interp1d(x, y, kind="cubic")
xx = np.linspace(0, 10, 200)
plt.figure()
plt.plot(x, y, 'o')
plt.plot(xx, f_cub(xx))
plt.title("Interpolation 1D (cubique)")
plt.xlabel("x"); plt.ylabel("y")
plt.tight_layout()



---
## 5) `scipy.stats` — Statistiques & tests


In [ ]:

from scipy import stats
rng = np.random.default_rng(123)
sample = rng.normal(loc=0.0, scale=1.0, size=200)
mu = sample.mean()
sd = sample.std(ddof=1)
ci = stats.t.interval(confidence=0.95, df=len(sample)-1, loc=mu, scale=sd/np.sqrt(len(sample)))
mu, sd, ci


In [ ]:

# t-test & comparaison de moyennes
g1 = rng.normal(0.2, 1.0, 120)
g2 = rng.normal(0.5, 1.1, 130)
t2, p2 = stats.ttest_ind(g1, g2, equal_var=False)
t2, p2



---
## 6) `scipy.signal` & `scipy.fft`


In [ ]:

from scipy.signal import butter, filtfilt
from scipy import fft

fs = 200.0
t = np.arange(0, 2.0, 1/fs)
x = np.sin(2*np.pi*5*t) + 0.5*np.sin(2*np.pi*30*t) + 0.8*np.random.default_rng(0).normal(0,1,t.size)

b, a = butter(N=4, Wn=10/(0.5*fs), btype='low')
y = filtfilt(b, a, x)

plt.figure()
plt.plot(t, x)
plt.title("Signal bruité")
plt.xlabel("t (s)"); plt.ylabel("x(t)")
plt.tight_layout()

plt.figure()
plt.plot(t, y)
plt.title("Signal filtré (10 Hz)")
plt.xlabel("t (s)"); plt.ylabel("y(t)")
plt.tight_layout()

Y = fft.rfft(y)
F = fft.rfftfreq(t.size, d=1/fs)
plt.figure()
plt.plot(F, np.abs(Y)/len(Y))
plt.title("Spectre amplitude")
plt.xlabel("Fréquence (Hz)"); plt.ylabel("Amplitude")
plt.tight_layout()



---
## 7) `scipy.spatial` — KD‑Tree & distances


In [ ]:

from scipy.spatial import cKDTree, distance

rng = np.random.default_rng(123)
pts = rng.normal(size=(200, 2))
tree = cKDTree(pts)
dists, idx = tree.query(pts[0], k=5)
D = distance.pdist(pts[:10], metric="euclidean")
dists[:3], idx[:3], D[:5]



---
## 8) `scipy.ndimage` — imagerie N-D


In [ ]:

from scipy import ndimage as ndi

rng = np.random.default_rng(0)
img = (rng.random((200, 200)) > 0.97).astype(float)
img = ndi.gaussian_filter(img, sigma=1.0)

binary = img > (img.mean() + img.std())
labels, n = ndi.label(binary)

plt.figure()
plt.imshow(img, origin="lower")
plt.title("Image floutée")
plt.tight_layout()

plt.figure()
plt.imshow(binary, origin="lower")
plt.title("Seuillage")
plt.tight_layout()

n



---
## 9) 🎯 Mini‑projet — *Chaîne SciPy*
1) Filtrer un signal bruité (Butterworth)  
2) Identifier la fréquence dominante (FFT)  
3) Ajuster une sinusoïde via `optimize.minimize`  
4) Estimer un IC 95% pour l’amplitude (bootstrap)  
5) Tracer 3 figures séparées (brut, filtré, spectre) et afficher les paramètres


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy import fft, optimize, stats

rng = np.random.default_rng(123)
fs = 250.0
t = np.arange(0, 3.0, 1/fs)
true_f = 7.5
true_A = 2.0
true_phi = 0.4
x_clean = true_A*np.sin(2*np.pi*true_f*t + true_phi)
x = x_clean + 0.6*rng.normal(0, 1, size=t.size)

b, a = butter(4, 20/(0.5*fs), btype="low")
y = filtfilt(b, a, x)

Y = fft.rfft(y)
F = fft.rfftfreq(t.size, d=1/fs)
f0 = F[np.argmax(np.abs(Y))]

def model(params, t):
    A, phi = params
    return A*np.sin(2*np.pi*f0*t + phi)

def mse(params):
    return np.mean((y - model(params, t))**2)

res = optimize.minimize(mse, x0=[1.0, 0.0])
A_hat, phi_hat = res.x

def stat_func(data, axis):
    t, y = data
    def mseA(params):
        A, phi = params
        return np.mean((y - (A*np.sin(2*np.pi*f0*t + phi)))**2)
    r = optimize.minimize(mseA, x0=[1.0, 0.0])
    return r.x[0]

data = (t, y)
boot = stats.bootstrap((data,), stat_func, vectorized=False, n_resamples=200, paired=True, confidence_level=0.95, method="basic")
ci_A = boot.confidence_interval

plt.figure()
plt.plot(t, x)
plt.title("Signal brut")
plt.xlabel("t (s)"); plt.ylabel("x(t)")
plt.tight_layout()

plt.figure()
plt.plot(t, y)
plt.title("Signal filtré (low-pass)")
plt.xlabel("t (s)"); plt.ylabel("y(t)")
plt.tight_layout()

plt.figure()
plt.plot(F, np.abs(Y)/len(Y))
plt.title("Spectre (amplitude)")
plt.xlabel("Fréquence (Hz)"); plt.ylabel("Amplitude")
plt.tight_layout()

(A_hat, phi_hat, f0, (ci_A.low, ci_A.high))



---
## 📚 Ressources
- https://docs.scipy.org/doc/scipy/
